<a href="https://colab.research.google.com/github/canyue1111/TEST2/blob/master/faceCrop_emotion_check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      //await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
import dlib
from PIL import Image
from skimage import io
import matplotlib.pyplot as plt
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.models import load_model
import numpy as np
from datetime import datetime
import pandas as pd


df = pd.DataFrame(columns =['Customers ID', 'Time', 'Emotion','FileName'] )
 #(0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral)
 
emotion_class = ['Angry','Disgust','Fear', 'Happy', 'Sad','Surprise','Neutral']

model= load_model('ResNet-50.h5')

detector = dlib.get_frontal_face_detector()
CustomersID = 0
CustomerStatus = 0 # 0 means no facedetected, 1 means yes

In [ ]:
#Function to load the cropped image
def load_image(filename):
    img = load_img(filename, grayscale=False, target_size=(197, 197))
    #convert to array
    img = img_to_array(img)
    #reshape into a single sample with  channe 3
    img = img.reshape(1, 197, 197, 3)
    #prepare pixel data
    img = img.astype('float32')
    img = img / 255.0
    return img


In [ ]:
#crop the screenshot
def detect_faces(image):
    # Run detector and get bounding boxes of the faces on image.
    detected_faces = detector(image, 1)
    face_frames = [(x.left(), x.top(),
                    x.right(), x.bottom()) for x in detected_faces]

    return face_frames

In [ ]:
def check_Face_Exist(img_path):
  max_score = 0.99
  # Detect faces http://dlib.net/face_detector.py.html
  img = dlib.load_rgb_image(img_path)
  dets, scores, idx = detector.run(img, 1,-1)
  for i, d in enumerate(dets):
      #print("Detection {}, score: {}, face_type:{}".format(
      #    d, scores[i], idx[i]))
      if scores[i]> max_score:
        max_score = scores[i]
  if max_score > 1.0:
    return True
  else:
    print("NO FACE")
    return False


In [ ]:
update_ = True
import time
 
img_path = 'photo.jpg'

while update_:
  
  #Update each 5 seconds
  time.sleep(5)

  filename = take_photo()

  if check_Face_Exist(img_path):
    print("Face Detected")
    image = io.imread(img_path)
    detected_faces = detect_faces(image)

  # Crop faces and plot
    for n, face_rect in enumerate(detected_faces):
      face = Image.fromarray(image).crop(face_rect)

      now = datetime.now()
      today = now.strftime("%H:%M")
      #file name setting
      name = str(CustomersID) + "_" + str(today) + '.png'
      #save file
      plt.savefig(name)

      # emotion_class[np.argmax(model.predict(load_image("file/" + name)))]
      df= df.append(pd.Series([CustomersID, 
                               today,
                               emotion_class[np.argmax(model.predict(load_image(name)))],
                               name], index=df.columns ), ignore_index=True)
      if n == 5:
        update_ = False
    
  else:
    print("NofaceDetected")
    if CustomerStatus == 1:
      CustomersID = CustomersID + 1
      CustomerStatus == 0



KeyboardInterrupt: ignored

In [ ]:
df

,Customers ID,Time,Emotion,FileName
0,0,03:19,Neutral,0_03:19.png
1,0,03:19,Neutral,0_03:19.png
2,0,03:23,Neutral,0_03:23.png
3,0,03:23,Surprise,0_03:23.png
4,0,03:33,Neutral,0_03:33.png
5,0,03:33,Surprise,0_03:33.png
6,0,03:33,Fear,0_03:33.png
7,0,03:33,Sad,0_03:33.png
8,0,03:33,Neutral,0_03:33.png
9,0,03:33,Fear,0_03:33.png
